# Querying eLuxembourgensia
This notebook will illustrate how to use the eLuxembourgensia digital collection. 
First, we will request the user (you) to enter a name of a person. We will then query Wikidata for the person. 
If there are multiple entries, then the user will be requested to select the desired result. 
We will then use the birth and death dates of the selected person to query the eLuxembourgensia collection and display
the newspapers that were published during that person's lifetime.



In [37]:
# A function to fetch the Wikidata API
import requests

def fetch_wikidata(params):
    url = 'https://www.wikidata.org/w/api.php'
    try:
        return requests.get(url, params=params)
    except:
        return 'ERR'

In [38]:
# Request a name from the user
name = input("Enter a name to search for in Wikidata:")

Enter a name to search for in Wikidata: Paul Eyschen


In [39]:
# Build the wikidata parameters
params = {
        'action': 'wbsearchentities',
        'format': 'json',
        'search': name,
        'language': 'en'
    }

# Fetch the data from Wikidata
wikidata_results = fetch_wikidata(params)

# Convert the response to JSON
if wikidata_results!= 'ERR':
    wikidata_results = wikidata_results.json()

In [41]:
def selectFromDict(options):
    index = 0
    indexValidList = []
    print('Select an option:')
    for optionName in options:
        index = index + 1
        indexValidList.extend([options[optionName]])
        print(str(index) + ') ' + optionName)
    inputValid = False
    while not inputValid:
        inputRaw = input('Option: ')
        inputNo = int(inputRaw) - 1
        if inputNo > -1 and inputNo < len(indexValidList):
            selected = indexValidList[inputNo]
            #print('Selected ' +  name + ': ' + selected)
            inputValid = True
            break
        else:
            print('Please select a valid option number.')
    return selected

options= {}
for entry in wikidata_results['search']:
    label = entry['label']
    id = entry['id']
    options[label] = id

selected_id = selectFromDict(options)
for label, id in options.items():
    if id == selected_id:
        selected_entry = label
        break

Select an option:
1) Paul Eyschen
2) Monument Paul Eyschen


Option:  1


In [42]:
# Retrieve the birth and death date from wikidata for the given id

# Create parameters
params = {
            'action': 'wbgetentities',
            'ids':selected_id, 
            'format': 'json',
            'languages': 'en'
        }

# fetch the API
wikidata_selected_entry = fetch_wikidata(params)

# Convert the response to JSON
if wikidata_selected_entry != 'ERR':
    wikidata_selected_entry = wikidata_selected_entry.json()


In [43]:
# P569 = birth date
birthdate = wikidata_selected_entry["entities"][selected_id]["claims"]["P569"][0]["mainsnak"]["datavalue"]["value"]["time"]
# parse out the date as yyyy-mm-dd
birthdate = birthdate[1:11]

'1841-09-09'

In [44]:
# P570 = death date
deathdate = wikidata_selected_entry["entities"][selected_id]["claims"]["P570"][0]["mainsnak"]["datavalue"]["value"]["time"]
deathdate = deathdate[1:11]

'1915-10-12'

In [46]:
# get the BnL eLuxembourgensia collection
elux_collection = requests.get("https://viewer.eluxemburgensia.lu/api/viewer2/cms/digitalcollections")
elux_collection = elux_collection.json()

In [58]:
# select only those newspapers published between the birth date and death date of the person
print("Newspapers published during " + selected_entry + "'s lifetime (" + birthdate + ", " + deathdate + "):")

import pandas as pd
pd.set_option('display.max_rows', None)

filtered_newspapers = []
for newspaper in elux_collection["data"]:
    newspaper_dict = {}
    startdate = newspaper["startdate"]
    try:
        enddate = newspaper["enddate"]
    except:
        enddate = "9999-12-31"
    if startdate <= deathdate or enddate >= birthdate:
        newspaper_dict = {'Title': newspaper["title"],'Start Date': startdate, 'End Date': enddate, 'Link': "https://persist.lu/" + newspaper["ark"]}
        filtered_newspapers.append(newspaper_dict)

pd.DataFrame(filtered_newspapers, columns=["Title", "Start Date", "End Date", "Link"])

Newspapers published during Paul Eyschen's lifetime (1841-09-09, 1915-10-12):


,Title,Start Date,End Date,Link
0,Indépendance luxembourgeoise (L'),1871-10-01,1934-12-31,https://persist.lu/ark:70795/pwn53jptg5
1,Luxemburger Wort,1848-03-23,9999-12-31,https://persist.lu/ark:70795/81794hthg5
2,Action féminine (L'),1927-10-15,1940-01-08,https://persist.lu/ark:70795/4w3g896b8b
3,Arbeiter (Der),1878-04-03,1882-11-04,https://persist.lu/ark:70795/bsx8pmtp83
4,Arlequin (L'),1848-04-15,1848-05-10,https://persist.lu/ark:70795/gntnjh48r4
5,Arme Teufel (Der),1903-11-29,1929-08-31,https://persist.lu/ark:70795/vmgpr7hpx3
6,Avenir (L'),1868-04-21,1871-09-29,https://persist.lu/ark:70795/vs3q03dprt
7,Bulletin de la Société des naturalistes luxemb...,1907-03-31,9999-12-31,https://persist.lu/ark:70795/h361mqns82
8,Bürger- und Beamten-Zeitung,1898-12-28,1916-12-28,https://persist.lu/ark:70795/1zvm975xd7
9,Cäcilia,1862-01-01,1871-12-01,https://persist.lu/ark:70795/2ws49ft71x
